# Alumno: Gerardo de Miguel González

##  Práctica Remuestreo

### Carga y limpieza de los datos

In [2]:
#::GMG::Cargamos los datos
data <- read.csv(file = 'm1965-stats-ii-datos-practica-remuestreo.csv',header = TRUE,sep = ',')

In [3]:
#::GMG::La columna de de fechas (date) está en un formato no compatible con el tipo de fecha
#       DD/MM/YYYY adecuado para una indexación posterior que vamos a necesitar para la selección
#       de los parámetros AVG.Cond. (Conductividad) y AVG.Salinity. (Salinidad) de 2014 ...
#       Ajustamos las fechas
data$date <- as.Date(x = data$date, format='%d/%m/%Y %H:%M')

In [4]:
#::GMG::Vamos a extraer los datos que necesitamos para el remuestreo
myvars <- names(data) %in% c("date", "AVG.Temp.")
data.processed <- data[myvars]
head(data.processed)

date,AVG.Temp.
2014-01-01,3.997746
2014-01-02,4.057667
2014-01-03,4.133612
2014-01-04,4.188685
2014-01-05,4.346172
2014-01-06,4.392661


In [5]:
#::GMG::Ahora vamos a separar los datos de "verano" de 2014 y 2015
# https://stackoverflow.com/questions/9407622/subsetting-a-dataframe-for-a-specified-month-and-year
data.processed.verano <- subset(
           x = data.processed, 
           #data.processed$date >= as.Date('2014-07-01') & 
           #data.processed$date <= as.Date('2014-09-31')
           format.Date(data.processed$date, '%m') >= '07' &
           format.Date(data.processed$date, '%m') <= '09'
           )
head(data.processed.verano)

,date,AVG.Temp.
182,2014-07-01,15.41396
183,2014-07-02,14.70326
184,2014-07-03,14.76020
185,2014-07-04,15.12685
186,2014-07-05,15.19523
187,2014-07-06,15.24690


In [40]:
#::GMG::Ahora selecciono y limpio el vector (muestra) de temperatura de 2014 y 2015
#       Hay que limpiar los NaNs con na.omit()
# https://www.statmethods.net/input/missingdata.html
x.verano <- na.omit(data.processed.verano$AVG.Temp.)
paste('Tamaño de la muestra:',length(x.verano))

[1] "Tamaño de la muestra: 182"

### Media y desviación muestral

**::GMG::** Cálculo de la media y deviación estándar muestral (estadísticos/estimadores)

In [7]:
#::GMG::Usamos las funciones que nos proporciona R
media.x.verano <- mean(x.verano)
sd.x.verano <- sd(x.verano)
paste('Media:',round(media.x.verano,3),' - Desv. Estándar:', round(sd.x.verano,3))

[1] "Media: 18.066  - Desv. Estándar: 1.971"

### Bootstrap

Crea una función que reciba un vector de números ‘x’ (la muestra original) y un número
natural N, que genere una matriz que tenga N columnas siendo cada una de estas, una de las
muestras de bootstrap.

In [53]:
bootstrap <- function(x, N) {
    resamples <- lapply(1:N, function(i) sample(x, replace=T))
    r.mean <- sapply(resamples, mean)
    std.err <- sqrt(var(r.mean))
    list(std.err=std.err, resamples=resamples, means=r.mean)
}

In [23]:
b.mean_m <- function(x, N) {
    resamples <- lapply(1:N, function(i) sample(x, replace=T))
    return (matrix(unlist(resamples), ncol = length(resamples), byrow = FALSE))
}

In [24]:
b.mean_m_2 <- function(x, N) {
    resamples <- sapply(1:N, function(i) sample(x, replace=T))
    return (resamples)
}

In [54]:
b <- bootstrap(x = x.verano, 10)

In [55]:
b$std.err

[1] 0.117986

In [39]:
sqrt(sum((b$means - media.x.verano)^2)/length(b$resamples))

[1] 0.1576252

In [64]:
length(x.verano)

[1] 182

In [26]:
b_m <- b.mean_m(x.verano,10)

In [65]:
b_m_2 <- b.mean_m_2(x.verano,10)
print(b_m_2)

           [,1]     [,2]     [,3]     [,4]     [,5]     [,6]     [,7]     [,8]
  [1,] 17.99565 20.06201 18.06934 17.38163 18.53607 17.69168 19.85151 16.17347
  [2,] 17.69168 15.75357 14.54151 14.76020 15.29014 18.61168 20.19910 18.32746
  [3,] 17.93256 18.13736 18.56732 19.56717 18.33920 19.56717 17.77316 19.28838
  [4,] 17.79244 18.46036 18.33920 17.55973 17.88988 17.59175 19.41715 17.02871
  [5,] 18.56732 17.99077 17.77316 15.24690 21.06204 20.96890 18.88433 19.17337
  [6,] 19.52835 19.65550 18.09391 20.06201 17.60939 17.94831 16.17347 17.13531
  [7,] 24.12338 17.88988 19.21925 18.74176 18.43866 19.56717 19.54617 16.39321
  [8,] 18.43759 18.58149 20.70701 19.86110 15.41396 16.30219 19.23792 18.56732
  [9,] 22.89975 17.02694 19.40751 15.05532 16.34598 17.11487 15.14063 19.82364
 [10,] 18.30479 15.32608 22.35945 18.12732 16.49459 18.43866 18.30596 20.25232
 [11,] 16.39321 16.06334 17.74249 15.68746 16.34598 17.74249 18.06934 15.86895
 [12,] 15.41396 19.86110 18.32765 19.35242 18.58149 

In [28]:
typeof(b)

[1] "list"

In [29]:
dim(b)

NULL

In [30]:
length(b)

[1] 3

In [31]:
typeof(b_m)

[1] "double"

In [32]:
dim(b_m)

[1] 182  10

In [33]:
typeof(b_m_2)

[1] "double"

In [34]:
dim(b_m_2)

[1] 182  10